Turn a Tithe map into patches for model to predict on (256x256 pixel patches with padding):
***
*Running with base but maybe make a different environement for this.*

In [1]:
from PIL import Image
import numpy as np
from patchify import patchify
import os
Image.MAX_IMAGE_PIXELS = None # disable protection from potential decompression bomb DOS attack
map_name = "Selworthy"
image = Image.open(f"../Full/{map_name}.tif")
#display(image)
image_array = np.array(image)
print(image_array.shape)

(15592, 16534, 3)


In [2]:
patch_size = 256
# Calculate padding needed
h, w, c = image_array.shape
pad_h = (patch_size - h % patch_size) % patch_size
pad_w = (patch_size - w % patch_size) % patch_size
print(f"Padding needed: height={pad_h}, width={pad_w}")
# Pad the image
padded_image = np.pad(image_array, 
                     ((0, pad_h), (0, pad_w), (0, 0)), 
                     mode='constant', 
                     constant_values=0)
# show the image with padding:
#Image.fromarray(padded_image.astype(np.uint8)).show()

Padding needed: height=24, width=106


In [3]:
patches=patchify(padded_image,(patch_size,patch_size,3),step=patch_size)
print(patches.shape)

(61, 65, 1, 256, 256, 3)


In [4]:
output_dir = f"{map_name}_image_patches_{patch_size}"

#patch_height, patch_width, _ = patches.shape[1:4]
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):
        patch = patches[i, j, 0]
        patch_img = Image.fromarray(patch.astype(np.uint8))
        patch_filename = f"patch_{i}_{j}.png"
        patch_img.save(os.path.join(output_dir, patch_filename))
